In [1]:
import nltk
import os
import re
import math
import operator
import json
import tqdm

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
def lemmatize_words(words):
    lemmatized_words = []
    for word in words:
       lemmatized_words.append(wordlemmatizer.lemmatize(word))
    return lemmatized_words
# def stem_words(words):
#     stemmed_words = []
#     for word in words:
#        stemmed_words.append(stemmer.stem(word))
    return stemmed_words
def remove_special_characters(text):
    regex = r'[^a-zA-Z0-9\s]'
    text = re.sub(regex,'',text)
    return text
def freq(words):
    words = [word.lower() for word in words]
    dict_freq = {}
    words_unique = []
    for word in words:
       if word not in words_unique:
           words_unique.append(word)
    for word in words_unique:
       dict_freq[word] = words.count(word)
    return dict_freq
def pos_tagging(text):
    pos_tag = nltk.pos_tag(text.split())
    pos_tagged_noun_verb = []
    for word,tag in pos_tag:
        if tag == "NN" or tag == "NNP" or tag == "NNS" or tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
             pos_tagged_noun_verb.append(word)
    return pos_tagged_noun_verb
def tf_score(word,sentence):
    freq_sum = 0
    word_frequency_in_sentence = 0
    len_sentence = len(sentence)
    for word_in_sentence in sentence.split():
        if word == word_in_sentence:
            word_frequency_in_sentence = word_frequency_in_sentence + 1
    tf =  word_frequency_in_sentence/ len_sentence
    return tf
def idf_score(no_of_sentences,word,sentences):
    no_of_sentence_containing_word = 0
    for sentence in sentences:
        sentence = remove_special_characters(str(sentence))
        sentence = re.sub(r'\d+', '', sentence)
        sentence = sentence.split()
        sentence = [word for word in sentence if word.lower() not in Stopwords and len(word)>1]
        sentence = [word.lower() for word in sentence]
        sentence = [wordlemmatizer.lemmatize(word) for word in sentence]
        if word in sentence:
            no_of_sentence_containing_word = no_of_sentence_containing_word + 1
    idf = math.log10(no_of_sentences/no_of_sentence_containing_word)
    return idf
def tf_idf_score(tf,idf):
    return tf*idf
def word_tfidf(dict_freq,word,sentences,sentence):
    word_tfidf = []
    tf = tf_score(word,sentence)
    idf = idf_score(len(sentences),word,sentences)
    tf_idf = tf_idf_score(tf,idf)
    return tf_idf
def sentence_importance(sentence,dict_freq,sentences):
     sentence_score = 0
     sentence = remove_special_characters(str(sentence)) 
     sentence = re.sub(r'\d+', '', sentence)
     pos_tagged_sentence = [] 
     no_of_sentences = len(sentences)
     pos_tagged_sentence = pos_tagging(sentence)
     for word in pos_tagged_sentence:
          if word.lower() not in Stopwords and word not in Stopwords and len(word)>1: 
                word = word.lower()
                word = wordlemmatizer.lemmatize(word)
                sentence_score = sentence_score + word_tfidf(dict_freq,word,sentences,sentence)
     return sentence_score


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ekansh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/ekansh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ekansh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
# writefile = open('summary.txt', 'w')       

def func (text):
    tokenized_sentence = sent_tokenize(text)
    text = remove_special_characters(str(text))
    text = re.sub(r'\d+', '', text)
    tokenized_words_with_stopwords = word_tokenize(text)
    tokenized_words = [word for word in tokenized_words_with_stopwords if word not in Stopwords]
    tokenized_words = [word for word in tokenized_words if len(word) > 1]
    tokenized_words = [word.lower() for word in tokenized_words]
    tokenized_words = lemmatize_words(tokenized_words)
    word_freq = freq(tokenized_words)
    input_user = 50
    no_of_sentences = int((input_user * len(tokenized_sentence))/100)
    # print(no_of_sentences)
    c = 1
    sentence_with_importance = {}
    # print(len(tokenized_sentence))
    # print('\n')
    for sent in tokenized_sentence:
        # print(c)
        # print('\n')
        sentenceimp = sentence_importance(sent,word_freq,tokenized_sentence)
        sentence_with_importance[c] = sentenceimp
        c = c+1
    # print(c)
    sentence_with_importance = sorted(sentence_with_importance.items(), key=operator.itemgetter(1),reverse=True)
    cnt = 0
    summary = []
    sentence_no = []
    # print(cnt)

    for word_prob in sentence_with_importance:
        if cnt < no_of_sentences:
            sentence_no.append(word_prob[0])
            cnt = cnt+1
        else:
            break
    sentence_no.sort()
    cnt = 1
    for sentence in tokenized_sentence:
        if cnt in sentence_no:
            summary.append(sentence)
        cnt = cnt+1
    summary = " ".join(summary)
    # print("\n")
    # print("Summary:")
    # print(summary)
    return summary


# read the json file
with open('biolaysumm2024_data/eLife_val.jsonl') as f:
    data = f.readlines()
    # print(data)
    index = 0
    finalarr = []

    for i in tqdm.tqdm(data):
        if index == 10:
            break
        ni = json.loads(i)
        text = ni['article']
        # take the string only till we encounter the second '\n' and remove the rest
        text = text[:text.find('\n', text.find('\n')+1)]
        stri = func(text)
        finalarr.append(stri)
        index += 1

  4%|▍         | 10/241 [00:16<06:18,  1.64s/it]


In [6]:
# write to a file with each summary in a new line
outF = open('elifeabsintrotfidf.txt',"w")
for line in finalarr:
  print(line)
  outF.write(line)
  outF.write("\n")

Cell-fate reprograming is at the heart of development , yet very little is known about the molecular mechanisms promoting or inhibiting reprograming in intact organisms . elegans germline , reprograming germ cells into somatic cells requires chromatin perturbation . Here , we describe that such reprograming is facilitated by GLP-1/Notch signaling pathway . Through a combination of genetics , tissue-specific transcriptome analysis , and functional studies of candidate genes , we uncovered a possible explanation for this unexpected role of GLP-1/Notch . We propose that GLP-1/Notch promotes reprograming by activating specific genes , silenced by the Polycomb repressive complex 2 ( PRC2 ) , and identify the conserved histone demethylase UTX-1 as a crucial GLP-1/Notch target facilitating reprograming . Cell-fate decisions are controlled , on the one hand , by intercellular signaling and , on the other hand , by intrinsic mechanisms such as epigenetic chromatin modifications . The Notch sign

In [ ]:
# write to a file with each summary in a new line
outF = open('plosabsintrotfidf.txt',"w")
for line in finalarr:
  print(line)
  outF.write(line)
  outF.write("\n")

enterica subsp .
For example , wrongly rejecting a food item might see an animal missing a meal , but wrongly accepting a food item could see an animal ingesting poison .
elegans .
This framework uses 16S rRNA sequencing count data to learn a smoothly evolving Gaussian process .
Freeman characterized gamma activity in terms of its topography , frequency range , and relationship to unitary activity and behavior .
Thus , underreplication provides a facile readout for late replication initiation and delayed fork progression .
, 2019; van den Akker et al .
Finally , in a murine model of house dust mite ( HDM ) -induced allergic airway disease , we demonstrated that anxa1 protein expression increased and anxa1 mRNA was significantly induced in lung tissue following HDM exposure .
, 2014; van Moorsel et al .
, 1992; van Doorn et al .
In male mammals , the Y-chromosomal transcription factor ( TF ) SRY activates a gene regulatory network promoting testis formation and repressing ovary formatio